In [1]:
import collections
from IPython.display import display
from tensorflow import keras
import pandas as pd
from random import sample
import numpy as np
import seaborn as sns
import os
from tqdm import trange
from random import sample
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense,LSTM,Dropout
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
import tensorflow as tf
from tensorflow.keras.metrics import MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,MeanAbsolutePercentageError
import nest_asyncio
from time import time
import sys
import tensorflow_federated as tff
import uuid



2023-04-18 16:55:02.441578: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [1]:

def get_sample(dataset,k=0.1):
    return sample(dataset,int(k * len(dataset)))



In [3]:


def create_tf_dataset_for_client_fn(path): # path is name of country the 'federator'
    def batch_format_fn(element):

        return collections.OrderedDict(
            x=element['x'],
            y=element['y'],)
    #
    data_x= np.load(f'{path}data_X.npy')
    data_y= np.load(f'{path}data_Y.npy')

    data = collections.OrderedDict((('y', data_y), ('x', data_x)))
    dataset = tf.data.Dataset.from_tensor_slices(data)
    return dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [4]:


def get_training_Q(Q):
    size = int(Q*len(train_data.client_ids))
    sampled_clients = np.random.choice(
            train_data.client_ids,
            size=size,
            replace=False)
    
    sampled_train_data = [
            train_data.create_tf_dataset_for_client(client)
            for client in sampled_clients
        ]
    
    return sampled_train_data

def get_test_Q(Q):
    size = int(Q*len(test_data.client_ids))
    sampled_clients = np.random.choice(
            test_data.client_ids,
            size=size,
            replace=False)
    
    ids = [_id.split('/')[-3] for _id in sampled_clients]
    print(ids)
    
    sampled_test_data = [
            test_data.create_tf_dataset_for_client(client)
            for client in sampled_clients
        ]
    return sampled_test_data

In [5]:
def get_eval_dataset():
    test_data_paths = glob('../Dataset/*/test/*/scaled/')
    
    dataset_test= []
    for path in test_data_paths:
        
        d = dict(X= np.load(f'{path}data_X.npy'),
                 y= np.load(f'{path}data_Y.npy'))    
        dataset_test.append(d)
        
    dataset = {}
    
    dataset['X_test'] = np.concatenate([data['X'] for data in dataset_test],axis=0)
    dataset['y_test'] = np.concatenate([data['y'] for data in dataset_test],axis=0)
    
    return dataset

def evaluate_model(model):
    X_test= eval_dataset['X_test']
    y_test = eval_dataset['y_test']
    
    ev = model.evaluate(x=X_test,y=y_test,batch_size=2048,return_dict=True)
    result  = pd.DataFrame(ev,index=[0])
    return result

def from_federated_to_keras(state):
    federated_model = base_model()
    federated_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1.0),
                            loss=tf.keras.losses.MeanSquaredError(),
                            metrics=[MeanAbsoluteError(), MeanSquaredError(), RootMeanSquaredError(),
                                    MeanAbsolutePercentageError() 
                                    ])
    state.model.assign_weights_to(model=federated_model)
    return federated_model
    
def evaluate_federated_model(state):
    federated_model = from_federated_to_keras(state)

    return evaluate_model(federated_model)


In [6]:
def get_model_in_text(path,model_fn):
    m = model_fn()
    with open(f'{path}model_report.txt','w') as fh:
        m.summary(print_fn=lambda x: fh.write(x + '\n'))

In [9]:
#GET-DATASETS
from glob import glob
from random import sample
import pandas as pd
import tensorflow as tf
import collections
# PREFETCH_BUFFER = 512
# BATCH_SIZE = 512
# NUM_EPOCHS = 10
PREFETCH_BUFFER = 1024
BATCH_SIZE = 1024
NUM_EPOCHS = 5

df = pd.read_csv("../../../data/df_testing.csv")
# train_data_paths =glob('.')
# test_data_paths = glob('/scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Dataset/*/test/*/scaled/')



# train_data_paths = get_sample(train_data_paths)
# test_data_paths = get_sample(test_data_paths)

def create_tf_dataset_for_client_fn(df, federation_level): # path is name of country the 'federator'
    def batch_format_fn(element):
        return collections.OrderedDict(
            x=element['x'],
            y=element['y'],)
    
    df = df[df["federation_level"]==federation_level]

    data_x = df.x.values
    data_y = df.y.values

    data = collections.OrderedDict((('y', data_y), ('x', data_x)))
    dataset = tf.data.Dataset.from_tensor_slices(data)
    return dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

df_tf_sim =create_tf_dataset_for_client_fn(df, "germany")

train_data = tff.simulation.datasets.ClientData.from_clients_and_fn(
        client_ids=train_data_paths,
        create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn
    )

test_data = tff.simulation.datasets.ClientData.from_clients_and_fn(
        client_ids=test_data_paths,
        create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn
    )

example_dataset = train_data.create_tf_dataset_for_client(
        train_data.client_ids[0]
    )

eval_dataset = get_eval_dataset()


NameError: name 'tff' is not defined

In [10]:
df_tf_sim

<PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('y', TensorSpec(shape=(None,), dtype=tf.int64, name=None))])>

In [8]:

def get_iterative_process(noise,n_clients=0):
    if noise == 0.0:
        iterative_process = tff.learning.build_federated_averaging_process(
            model_tff,
            client_optimizer_fn=lambda: keras.optimizers.Adam(),
            server_optimizer_fn=lambda: keras.optimizers.SGD(learning_rate=1),
            use_experimental_simulation_loop=True)
    else:
        aggregation_factory = tff.learning.model_update_aggregator.dp_aggregator(noise, n_clients)
        iterative_process = tff.learning.build_federated_averaging_process(
            model_tff,
            client_optimizer_fn=lambda: keras.optimizers.Adam(),
            server_optimizer_fn=lambda: keras.optimizers.SGD(learning_rate=1),
            model_update_aggregation_factory=aggregation_factory,
            use_experimental_simulation_loop=True)
    
    return iterative_process

In [9]:
def base_model():
    model = Sequential()
    model.add(LSTM(100,return_sequences=False,input_shape=(24,1)))
    model.add(Dense(24))
    return model

In [10]:
def model_tff():
    model = base_model()
    return tff.learning.from_keras_model(
      model,
      input_spec=train_data.element_type_structure,
      loss=keras.losses.MeanSquaredError(),
      metrics=[
            MeanAbsoluteError(), MeanSquaredError(), RootMeanSquaredError(),
                                    MeanAbsolutePercentageError() 
              ])



In [11]:
def save_all(results,state,n_rounds,config,unique_id):
    results.reset_index(inplace=True,drop=True)

    centralized_model = from_federated_to_keras(state=state)
    path = f'/scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/{unique_id}/{n_rounds}/'

    config['round'] = n_rounds
    

    if not os.path.exists(path):
        print(f'Saving in {path}')
        os.makedirs(path)
        pd.DataFrame(config,index=[0]).to_csv(f'{path}config.csv')
        get_model_in_text(path,base_model)
        results.to_csv(f'{path}results.csv')
        centralized_model.save(f'{path}model')

In [12]:
total_clients = len(train_data_paths)
n_clients = 70
Noise = 0.48
Q = n_clients/total_clients


In [13]:
config = {'Q':Q,
          'clients_per_round':int(n_clients),
          'total_clients':len(train_data_paths),
          'noise': Noise,
          'batch_size':int(BATCH_SIZE),
          'eval_client_size':len(glob('../Dataset/*/test/*/scaled/')),
          'internal_epochs':int(NUM_EPOCHS)}

In [14]:
print(config)

{'Q': 0.2028985507246377, 'clients_per_round': 70, 'total_clients': 345, 'noise': 0.48, 'batch_size': 1024, 'eval_client_size': 890, 'internal_epochs': 5}


In [15]:
tff.backends.native.set_local_execution_context(clients_per_thread=1)

In [16]:
iterative_process = get_iterative_process(Noise,n_clients)
evaluation = tff.learning.build_federated_evaluation(model_tff,use_experimental_simulation_loop=True)


In [17]:
nest_asyncio.apply()
state = iterative_process.initialize()  
NUM_ROUNDS = 125
results = pd.DataFrame()
unique_id = str(uuid.uuid4())[0:8]


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2023-04-18 16:55:58.616773: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-04-18 16:55:58.628652: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2600000000 Hz


In [18]:
t = trange(1,NUM_ROUNDS, desc='', leave=True)
        
for round_num in t:
    

    start = time()
    
    train_set = get_training_Q(Q)

    state, metrics = iterative_process.next(state, train_set)
    
    if round_num % 25 == 0:
        metrics['train']['round_number'] = int(round_num)
        metrics['train']['time'] = time()-start
        metrics_train_df = pd.DataFrame(metrics['train'],index=[0])
        sys.stdout.flush()
        
        if Noise != 0.0:
            print(state.delta_aggregate_state)

        print(f'Train:{metrics_train_df["mean_absolute_percentage_error"].iloc[0]}')
        
        metrics_test_df = evaluate_federated_model(state)
        metrics_test_df['round_number']= round_num
        metrics_train_df = metrics_train_df.merge(metrics_test_df, on='round_number', suffixes=('_train','_test'))
        results = pd.concat([results,metrics_train_df])
       
        save_all(results=results,state= state,n_rounds=round_num,config=config,unique_id=unique_id)
       
        

  0%|          | 0/124 [00:00<?, ?it/s]2023-04-18 16:59:52.451641: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-04-18 16:59:52.899884: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
 19%|█▉        | 24/124 [55:26<3:23:05, 121.86s/it]

OrderedDict([('zeroing_norm', (_GlobalState(current_estimate=3.1622777, target_quantile=0.98, learning_rate=2.3025851, below_estimate_state=()), ())), ('inner_agg', (_GlobalState(numerator_state=_GlobalState(noise_multiplier=0.48113248, sum_state=_GlobalState(l2_norm_clip=0.4943598, stddev=0.23785256), quantile_estimator_state=_GlobalState(current_estimate=0.4943598, target_quantile=0.5, learning_rate=0.2, below_estimate_state=_GlobalState(numerator_state=_GlobalState(l2_norm_clip=0.5, stddev=3.5), denominator=70.0))), denominator=70.0), ())), ('zeroed_count_agg', ())])
Train:55.05063247680664


2023-04-18 17:53:42.278568: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-04-18 17:53:43.475120: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8004


8453/8453 [==============================] - 27s 3ms/step - loss: 2.9891e-04 - mean_absolute_error: 0.0099 - mean_squared_error: 2.9891e-04 - root_mean_squared_error: 0.0173 - mean_absolute_percentage_error: 69.8990
Saving in /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/25/


2023-04-18 17:54:10.968084: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


...

INFO:tensorflow:Assets written to: /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/75/model/assets


INFO:tensorflow:Assets written to: /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/75/model/assets
 80%|███████▉  | 99/124 [3:37:37<53:38, 128.73s/it]  

OrderedDict([('zeroing_norm', (_GlobalState(current_estimate=0.100000046, target_quantile=0.98, learning_rate=2.3025851, below_estimate_state=()), ())), ('inner_agg', (_GlobalState(numerator_state=_GlobalState(noise_multiplier=0.48113248, sum_state=_GlobalState(l2_norm_clip=0.25863373, stddev=0.12443709), quantile_estimator_state=_GlobalState(current_estimate=0.25863373, target_quantile=0.5, learning_rate=0.2, below_estimate_state=_GlobalState(numerator_state=_GlobalState(l2_norm_clip=0.5, stddev=3.5), denominator=70.0))), denominator=70.0), ())), ('zeroed_count_agg', ())])
Train:34.83448028564453
8453/8453 [==============================] - 24s 3ms/step - loss: 1.5793e-04 - mean_absolute_error: 0.0067 - mean_squared_error: 1.5793e-04 - root_mean_squared_error: 0.0126 - mean_absolute_percentage_error: 45.2697
Saving in /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/100/


INFO:tensorflow:Assets written to: /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/100/model/assets


INFO:tensorflow:Assets written to: /scratch/users/jodelgado/classifier_forecaster/forecasting_classifier_privacy/Federated_Learning/results/experiments/1000_rounds/4ffbd676/100/model/assets
100%|██████████| 124/124 [4:32:03<00:00, 129.64s/it]


In [19]:
#  MAPE = 25.108067
# Q	0.005794
# clients_per_round	10
# total_clients	1726
# noise	0.10
# batch_size	512
# internal_epochs	10
# Optimizer ADAM
# 100 rounds 

In [20]:
results

,mean_absolute_error_train,mean_squared_error_train,root_mean_squared_error_train,mean_absolute_percentage_error_train,loss_train,round_number,time,loss_test,mean_absolute_error_test,mean_squared_error_test,root_mean_squared_error_test,mean_absolute_percentage_error_test
0,0.006792,0.000192,0.013864,55.050632,0.000192,25,132.741692,0.000299,0.009923,0.000299,0.017289,69.899033
1,0.005065,0.000095,0.009732,34.695339,0.000095,50,124.682097,0.000218,0.008156,0.000218,0.014752,55.447880
2,0.005063,0.000090,0.009490,33.979881,0.000090,75,121.416200,0.000187,0.007681,0.000187,0.013659,56.774860
3,0.004827,0.000083,0.009105,34.834480,0.000083,100,120.216939,0.000158,0.006653,0.000158,0.012567,45.269745
